In [ ]:
import tensorflow as tf 

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path 
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [ ]:
from dogCatClassifier.constants import *
from dogCatClassifier.utils import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = self.config.training.trained_model_path,
            training_data = "artifacts/data_ingestion/PetImages", 
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        data_generator_kwargs = dict(
            rescale= 1./255,
            validation_split=0.20
        )
        
        data_flow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset="validation",
            shuffle=False,
            **data_flow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path) 
    
    def evaluation(self):
        self.model - self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e